In [1]:
from pathlib import Path
import json
import time

import django
import pandas as pd
import requests

django.setup()

from django.conf import settings
from django.db.models import F, Q, Case, When, BooleanField, Sum
from django.utils.timezone import timedelta, now

from reports.models import SpecificStopSalesRules, BookingsFinancialData
from hotel.models import Hotel, Chain, Provider
from clients.models import Credential

In [2]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.float_format", "{:,.2f}".format)

In [3]:
base_dir = Path(".").resolve()
data_dir = Path(".").resolve() / "data"

In [4]:
class StopSalesDataProcessor:
    def __init__(self):
        self.level_keys = ["ps", "mel", "cid", "bod", "dow", "room", "non", "hou", "mrk"]
        self.simple_keys = ["cp", "rat", "rel", "age"]
        self.date_columns = [
            "last_updated",
            "check_in_from",
            "check_in_to",
        ]
        self.credential_group = self.get_credential_group()
        self.credential_dict = self.get_credential_dict()

    def get_stopsales_as_df(self, rule_id=None, credential_id=None, provider_id=None):
        qs = self.get_stopsales_data(rule_id)
        df = pd.DataFrame(list(qs))
        df = self._process_dates(df)
        df = self._transform_level_fields(df)
        
        if rule_id:
            df = df.explode("credential_id").reset_index(drop=True)

        if rule_id and credential_id:
            df = df[df["credential_id"] == credential_id]
            df = df.explode("provider_id").reset_index(drop=True)

        if rule_id and credential_id and provider_id:
            df = df[
                (df["credential_id"] == credential_id) & (df["provider_id"] == provider_id)
            ]
            df = df.explode("hotel_id").reset_index(drop=True)
            
        return df

    def get_stopsales_data(self, rule_id=None):
        qs = SpecificStopSalesRules.objects.filter(active=True)
        
        if rule_id:
            qs = qs.filter(rule_id=rule_id)
            
        return self.annotate_rules(qs).values(
            "rule_id", "organization", "credential_level", "credential_id",
            "provider_level", "provider_id", "hotel_level", "hotel_id",
            "destination_level", "destination_id", "market_level",
            "market_id", "rate_type", "refundable", "last_updated",
            "check_in_level", "check_in_from", "check_in_to", "has_attributes"
        )

    def annotate_rules(self, qs):
        return qs.annotate(
            credential_level=F("levels_rules__cli__t"),
            credential_id=F("levels_rules__cli__l"),
            provider_level=F("levels_rules__prv__t"),
            provider_id=F("levels_rules__prv__l"),
            hotel_level=F("levels_rules__hot__t"),
            hotel_id=F("levels_rules__hot__l"),
            destination_level=F("levels_rules__dest__t"),
            destination_id=F("levels_rules__dest__l"),
            market_level=F("levels_rules__mrk__t"),
            market_id=F("levels_rules__mrk__l"),
            rate_type=F("levels_rules__rat"),
            refundable=F("levels_rules__cp"),
            last_updated=F("last_date"),
            check_in_level=F("levels_rules__cid__t"),
            check_in_from=F("levels_rules__cid__f"),
            check_in_to=F("levels_rules__cid__u"),
            has_attributes=Case(
                When(self._get_key_conditions(), then=True),
                default=False,
                output_field=BooleanField(),
            )
        )

    def _get_key_conditions(self):
        conditions = Q()
        for key in self.level_keys + self.simple_keys:
            field = (
                f"levels_rules__{key}__t__gt"
                if key in self.level_keys
                else f"levels_rules__{key}__gt"
            )
            conditions |= Q(**{field: 0})
        return conditions
    
    def _process_dates(self, df):
        df[self.date_columns] = df[self.date_columns].apply(pd.to_datetime)
        return df
    
    def _transform_level_fields(self, df):
        df["credential_id"] = df.apply(self._map_credential_group, axis=1)
        df["credential_id"] = df["credential_id"].apply(self._remove_inactive_and_veci)
        df["hotel_id"] = df["hotel_id"].apply(self._remove_invalid_hotel_ids)
        return df
    
    @staticmethod
    def get_credential_group():
        df = pd.read_json(data_dir / "credential_group.json")

        return dict(zip(df["id"].astype(str), df["credential_list"]))


    @staticmethod
    def get_credential_dict():
        return {
            credential["id"]: {
                "enabled": credential["enabled"],
                "region": credential["client__region_id"],
            }
            for credential in Credential.objects.select_related("client__region").values(
                "id", "enabled", "client__region_id"
            )
        }
    
    def _map_credential_group(self, row):
        return (
            self._expand_credentials(row["credential_id"])
            if row["credential_level"] == 2
            else row["credential_id"]
        )


    def _expand_credentials(self, credential_ids):
        credentials = []
        for group_id in credential_ids:
            credentials.extend(self.credential_group.get(group_id, []))

        return list(dict.fromkeys(credentials))
    
    def _remove_inactive_and_veci(self, credential_ids):
        def is_valid(credential_id):
            credential_info = self.credential_dict.get(credential_id)
            if not credential_info:
                return False

            enabled = credential_info["enabled"]
            region = credential_info["region"]

            return enabled and (region and "veci" not in region.lower())

        return [
            credential_id
            for credential_id in (credential_ids or [])
            if credential_id is not None and is_valid(credential_id)
        ]
    
    @staticmethod
    def _remove_invalid_hotel_ids(ids):
        if ids is None:
            return None

        return [id for id in ids if isinstance(id, str) and id.isdigit()]

In [5]:
class MetricsProcessor:
    def __init__(self):
        pass

    def get_booking_data(self, row):
        filters = self.build_filters(row)
        qs = (
            BookingsFinancialData.objects.filter(**filters)
            .exclude(credential__client__region__name__icontains="veci")
            .values()
        )
        return pd.DataFrame(list(qs))

    @staticmethod
    def build_filters(row):
        to_list = lambda value: value if isinstance(value, list) else [value]

        levels = {
            "credential_level": {
                0: {},
                1: {"credential_id__in": to_list(row["credential_id"])},
                2: {"credential_id__in": to_list(row["credential_id"])},
            },
            "provider_level": {
                0: {},
                1: {"provider_id__in": to_list(row["provider_id"])},
                2: {"provider__direct": False},
                3: {"provider__direct": True},
            },
            "hotel_level": {
                0: {},
                1: {"hotel__in": to_list(row["hotel_id"])},
                2: {"hotel__chain__in": to_list(row["hotel_id"])},
            },
            "destination_level": {
                0: {},
                1: {"hotel__city__in": to_list(row["destination_id"])},
                2: {"hotel__country__in": to_list(row["destination_id"])},
            },
            "market_level": {
                0: {},
                1: {"requested_market": to_list(row["market_id"])},
            },
            "check_in_level": {
                0: {},
                1: {
                    "check_in__gte": row["check_in_from"],
                    "check_in__lte": row["check_in_to"],
                },
            },
            "rate_type": {
                0: {},
                1: {"is_package_rate": False},
                2: {"is_package_rate": True},
            },
            "refundable": {
                0: {},
                1: {"is_no_refundable": True},
                2: {"is_no_refundable": False},
            },
        }

        filters = {
            "booking_date__date__gte": row["last_updated"] - timedelta(days=365),
        }

        filters.update({
            key: val
            for level, mapping in levels.items()
            if (row_value := row.get(level)) in mapping
            for key, val in mapping[row_value].items()
        })

        return filters

    def get_metrics(self, row):
        df = self.get_booking_data(row)

        result = {
            "revenue_last_12_months": 0,
            "average_per_week": 0,
            "revenue_post_rule": 0,
            "monthly_revenue_last_12_months": {},
        }

        if df.empty:
            return result

        df_last_12_months = df[df["booking_date"].dt.date <= row["last_updated"].date()]
        df_post_rule = df[df["booking_date"].dt.date > row["last_updated"].date()]

        result["revenue_last_12_months"] = df_last_12_months["net_agency"].sum()
        result["average_per_week"] = self._calculate_avg_revenue_per_week(
            df_last_12_months
        )
        result["revenue_post_rule"] = df_post_rule["net_agency"].sum()
        result["monthly_revenue_last_12_months"] = self._calculate_monthly_revenue(
            df_last_12_months, row["last_updated"]
        )

        return result

    def _calculate_avg_revenue_per_week(self, df):
        df["booking_date"] = df["booking_date"].dt.tz_localize(None)
        df["week"] = df["booking_date"].dt.to_period("W")
        revenue_per_week = df.groupby("week")["net_agency"].sum()
        return revenue_per_week.mean()
    
    def _calculate_monthly_revenue(self, df, last_updated):
        df["month"] = df["booking_date"].dt.to_period("M")
        all_months = pd.date_range(
            last_updated - pd.DateOffset(months=12), last_updated, freq="MS"
        ).to_period("M")
        monthly_revenue = (
            df.groupby("month")["net_agency"].sum().reindex(all_months, fill_value=0)
        )

        return {str(month): revenue for month, revenue in monthly_revenue.items()}

In [6]:
stopsales_data_processor = StopSalesDataProcessor()
metrics_processor = MetricsProcessor()

In [7]:
df_stopsales = stopsales_data_processor.get_stopsales_as_df()
df_filtered = df_stopsales[df_stopsales.credential_level == 2].copy()
display(df_filtered)

,rule_id,organization,credential_level,credential_id,provider_level,provider_id,hotel_level,hotel_id,destination_level,destination_id,market_level,market_id,rate_type,refundable,last_updated,check_in_level,check_in_from,check_in_to,has_attributes
603,672054,lgt,2,"[34764, 32914, 32947, 33023, 32991, 32896, 328...",1,"[PRUEBA, TESTZZ, TESTZZ2, VECTORZZ, VECTORZZZ,...",0,None,0,[],0,None,0,0,2025-01-17,0,NaT,NaT,False
833,395307,lgt,2,"[32775, 34020, 34030, 34041, 34045, 34201, 341...",1,"[SMD, BCONG]",1,"[5269, 1434]",0,[],0,None,0,0,2024-10-11,0,NaT,NaT,False
858,395353,lgt,2,"[32775, 34020, 34030, 34041, 34045, 34201, 341...",1,"[DNG, SMD, BCONG]",1,[89127],0,[],0,None,0,0,2024-03-19,0,NaT,NaT,False
1159,452433,lgt,2,"[32775, 34020, 34030, 34041, 34045, 34201, 341...",1,[BCONG],2,"[684, 464]",0,[],0,None,0,0,2024-04-29,0,NaT,NaT,False
1276,684697,lgt,2,"[3576, 3693, 4651, 5815, 6288, 20669, 20776, 2...",1,[BCONG],1,[45945],0,[],0,None,0,0,2024-10-09,0,NaT,NaT,False
1435,732835,lgt,2,"[32775, 34020, 34030, 34041, 34045, 34201, 341...",1,[SHS],2,[1293],0,[],0,None,1,0,2024-11-18,0,NaT,NaT,True
1565,736024,lgt,2,"[32775, 34020, 34030, 34041, 34045, 34201, 341...",1,[MILLENNIUM],0,None,0,[],0,None,0,0,2024-11-20,0,NaT,NaT,False
1800,446223,lgt,2,"[34764, 32914, 32947, 33023, 32991, 32896, 328...",1,"[DNG, DIN2, FRT, DIN, BCONG]",1,"[2109, 68951, 126799, 1148, 103487, 1152, 1441...",0,[],0,None,0,0,2024-04-22,0,NaT,NaT,False
2171,397982,lgt,2,"[32775, 34020, 34030, 34041, 34045, 34201, 341...",1,"[SMD, BCONG]",1,"[185034, 411304, 2872, 107683, 4362, 1343765, ...",0,[],0,None,0,0,2024-10-02,0,NaT,NaT,False
2500,749190,lgt,2,"[32775, 34020, 34030, 34041, 34045, 34201, 341...",1,"[DNG, DIN2, DIN, BCONG]",1,"[2540, 2478650, 3243, 265888, 265887]",0,[],0,None,0,0,2024-11-29,0,NaT,NaT,False


In [8]:
start_time = time.time()

df_filtered[
    [
        "revenue_last_12_months",
        "average_per_week",
        "revenue_post_rule",
        "monthly_revenue_last_12_months",
    ]
] = df_filtered.apply(
    lambda row: pd.Series(metrics_processor.get_metrics(row)), axis=1
)

end_time = time.time()
execution_time = end_time - start_time

print(f"Execution time: {execution_time:.4f} seconds")

Execution time: 0.9684 seconds


In [12]:
df_filtered.head()

,rule_id,organization,credential_level,credential_id,provider_level,provider_id,hotel_level,hotel_id,destination_level,destination_id,market_level,market_id,rate_type,refundable,last_updated,check_in_level,check_in_from,check_in_to,has_attributes,revenue_last_12_months,average_per_week,revenue_post_rule,monthly_revenue_last_12_months
603,672054,lgt,2,"[34764, 32914, 32947, 33023, 32991, 32896, 328...",1,"[PRUEBA, TESTZZ, TESTZZ2, VECTORZZ, VECTORZZZ,...",0,None,0,[],0,None,0,0,2025-01-17,0,NaT,NaT,False,"52,248.88","2,488.04",0.00,"{'2024-02': 26827.37, '2024-03': 2858.11, '202..."
833,395307,lgt,2,"[32775, 34020, 34030, 34041, 34045, 34201, 341...",1,"[SMD, BCONG]",1,"[5269, 1434]",0,[],0,None,0,0,2024-10-11,0,NaT,NaT,False,"35,048.59","1,348.02",0.00,"{'2023-11': 6351.52, '2023-12': 2346.77, '2024..."
858,395353,lgt,2,"[32775, 34020, 34030, 34041, 34045, 34201, 341...",1,"[DNG, SMD, BCONG]",1,[89127],0,[],0,None,0,0,2024-03-19,0,NaT,NaT,False,"17,024.33","1,309.56",0.00,"{'2023-04': 2535.38, '2023-05': 0.0, '2023-06'..."
1159,452433,lgt,2,"[32775, 34020, 34030, 34041, 34045, 34201, 341...",1,[BCONG],2,"[684, 464]",0,[],0,None,0,0,2024-04-29,0,NaT,NaT,False,"200,716.76","3,716.98",0.00,"{'2023-05': 14624.42, '2023-06': 10245.73, '20..."
1276,684697,lgt,2,"[3576, 3693, 4651, 5815, 6288, 20669, 20776, 2...",1,[BCONG],1,[45945],0,[],0,None,0,0,2024-10-09,0,NaT,NaT,False,"2,365.64",788.55,0.00,"{'2023-11': 0.0, '2023-12': 0.0, '2024-01': 0...."


In [13]:
start_time = time.time()

df_stopsales = stopsales_data_processor.get_stopsales_as_df()
df_stopsales[
    [
        "revenue_last_12_months",
        "average_per_week",
        "revenue_post_rule",
        "monthly_revenue_last_12_months",
    ]
] = df_stopsales.apply(
    lambda row: pd.Series(metrics_processor.get_metrics(row)), axis=1
)

end_time = time.time()
execution_time = end_time - start_time

print(f"Execution time: {execution_time:.4f} seconds")

Execution time: 81.0307 seconds


In [14]:
display(df_stopsales.head(20))

,rule_id,organization,credential_level,credential_id,provider_level,provider_id,hotel_level,hotel_id,destination_level,destination_id,market_level,market_id,rate_type,refundable,last_updated,check_in_level,check_in_from,check_in_to,has_attributes,revenue_last_12_months,average_per_week,revenue_post_rule,monthly_revenue_last_12_months
0,291912,lgt,1,"[32775, 3576, 11266, 11404, 11409, 11506, 1161...",3,None,2,[4853],0,[],1,[ES],0,0,2024-12-11,0,NaT,NaT,True,0.00,0.00,0.00,{}
1,509914,lgt,1,"[34020, 34030, 34041, 34511, 34140, 34142, 341...",1,[BCONG],1,"[625423, 3758660, 3936090, 556770, 2512893, 18...",0,[],0,None,0,0,2024-12-11,0,NaT,NaT,False,"23,115.69",963.15,0.00,"{'2024-01': 0.0, '2024-02': 549.32, '2024-03':..."
2,2001,lgt,1,"[3576, 11266, 11404, 11409, 11506, 11617, 1175...",3,None,1,[1157],0,[],1,"[FR, IT, NL, PL]",0,0,2024-12-11,0,NaT,NaT,True,0.00,0.00,0.00,{}
3,468559,lgt,1,"[34030, 11617, 3576, 11409, 4204, 11506, 11753...",1,"[ROI, ROIB2B]",2,[197],0,[],0,None,0,0,2024-12-11,0,NaT,NaT,False,"28,426.07","2,842.61",0.00,"{'2024-01': 0.0, '2024-02': 0.0, '2024-03': 90..."
4,395670,lgt,1,"[34030, 3576, 4204, 11409, 11506, 11617, 11753...",1,"[SMD, YPL, BCONG, PRSP]",1,"[3898206, 186059, 3977200, 993, 427711, 102639...",0,[],0,None,0,0,2024-12-17,0,NaT,NaT,True,"169,523.50","3,260.07","17,986.38","{'2024-01': 19377.93, '2024-02': 19431.66, '20..."
5,445980,lgt,1,"[34030, 3576, 4204, 11409, 11506, 11617, 11753...",1,[BCONG],1,[149014],0,[],0,None,0,0,2024-12-11,0,NaT,NaT,False,"5,468.34",607.59,0.00,"{'2024-01': 2225.4300000000003, '2024-02': 114..."
6,395622,lgt,1,"[34030, 3576, 4982, 11409, 11506, 11617, 11753...",3,None,2,[1593],0,[],0,None,0,0,2024-12-11,0,NaT,NaT,False,104.54,104.54,0.00,"{'2024-01': 0.0, '2024-02': 0.0, '2024-03': 0...."
7,394569,lgt,1,"[32775, 3576, 11266, 11404, 11409, 11506, 1161...",1,"[OMB, OMBP]",1,"[1298891, 33552, 3862588, 1073761]",0,[],0,None,0,0,2024-12-11,0,NaT,NaT,False,0.00,0.00,0.00,{}
8,1890,lgt,1,"[3576, 11266, 11404, 11409, 11506, 11617, 1175...",3,None,1,[89390],0,[],1,"[AT, DE, PL]",0,0,2024-12-11,0,NaT,NaT,True,0.00,0.00,0.00,{}
9,395328,lgt,1,"[3576, 11266, 11404, 11409, 11506, 11617, 1175...",3,None,1,"[2061, 809, 558, 497237, 3898, 417, 6255, 89019]",0,[],1,[ES],0,0,2024-12-11,0,NaT,NaT,True,0.00,0.00,0.00,{}


In [11]:
# df_stopsales = df_stopsales.apply(
#     lambda row: pd.Series({
#         **metrics_processor.get_metrics(row),
#         **metrics_processor.get_metrics(row)["monthly_revenue_last_12_months"]
#     }), axis=1
# )